[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ZdivP_IbfTWcrM6sBI_XG8N2do9Gc9qD?usp=sharing)

## Monitoring ControlFlow Agents with Portkey

[ControlFlow](https://github.com/PrefectHQ/ControlFlow) is a Python framework for building agentic AI workflows.

ControlFlow provides a structured, developer-focused framework for defining workflows and delegating work to LLMs, without sacrificing control or transparency



[Portkey](https://app.portkey.ai/) is the Control Panel for AI apps. With it's popular AI Gateway and Observability Suite, hundreds of teams ship reliable, cost-efficient, and fast apps.

Portkey brings production readiness to Langchain. With Portkey, you can

 - Connect to 150+ models through a unified API,
 - View 42+ metrics & logs for all requests,
 - Enable semantic cache to reduce latency & costs,
 - Implement automatic retries & fallbacks for failed requests,
 - Add custom tags to requests for better tracking and analysis and more.

In [ ]:
!pip install -qU portkey-ai controlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.2/562.2 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.5/863.5 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
import controlflow as cf
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from portkey_ai import createHeaders, PORTKEY_GATEWAY_URL

portkey_headers = createHeaders(api_key= userdata.get("PORTKEY_API_KEY"), ## Grab from https://app.portkey.ai/
                                provider="openai"
                                )

gpt3 = ChatOpenAI(model = "gpt-3.5-turbo-0125",
                api_key= userdata.get("OPENAI_API_KEY"),
                 base_url=PORTKEY_GATEWAY_URL,
                 default_headers=portkey_headers)

# create an agent to write a research report
author = cf.Agent(
    name="Deep Thought",
    instructions="Use a formal tone and clear language",
    model = gpt3
)


class ResearchTopic(BaseModel):
    title: str
    keywords: list[str]

In [ ]:
@cf.flow
def research_workflow() -> str:
    # Task 1: the default agent will work with the user to choose a topic
    topic = cf.Task(
        "Work with the user to come up with a research topic",
        result_type=ResearchTopic,
        user_access=True,
    )

    # Task 2: the default agent will create an outline based on the topic
    outline = cf.Task("Create an outline", context=dict(topic=topic))

    # Task 3: the author agent will write a first draft
    draft = cf.Task(
        "Write a first draft",
        context=dict(outline=outline),
        agents=[author]
    )

    return draft


# run the workflow
result = research_workflow()
print(result)

Output()

🤖 Agent: Hello! I'd be happy to help you come up with a research topic. Do you have any specific areas of interest
or fields you are passionate about? For example, technology, health, environment, social sciences, etc.?
Type your response:

Alfred Alder's life lies


Output()

Output()

...

Output()

Output()

Output()

Output()

The outline for the research on 'The Life and Legacy of Alfred Adler' has been successfully created.


## Observability with Portkey

By routing requests through Portkey you can track a number of metrics like - tokens used, latency, cost, etc.

<img src="data:image/png;base64, ..." alt="CrewAI Dashboard" height = "500">